In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

In [29]:
df = spark.read.parquet("data/final/merge_jugadores.parquet")
df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Born: float (nullable = true)
 |-- MP: float (nullable = true)
 |-- Starts: float (nullable = true)
 |-- Min: float (nullable = true)
 |-- Gls: double (nullable = true)
 |-- Ast: double (nullable = true)
 |-- YellowC: float (nullable = true)
 |-- RedC: float (nullable = true)
 |-- PrgC: double (nullable = true)
 |-- PrgP: double (nullable = true)
 |-- PrgR: double (nullable = true)
 |-- Fls: double (nullable = true)
 |-- Off: double (nullable = true)
 |-- Crosses: double (nullable = true)
 |-- Recov: double (nullable = true)
 |-- Aerialwon%: float (nullable = true)
 |-- Tack_Def_3rd: double (nullable = true)
 |-- Tack_Mid_3rd: double (nullable = true)
 |-- Tack_Att_3rd: double (nullable = true)
 |-- Tkl%: float (nullable = true)
 |-- Tkl: double (nullable = true)
 |-- TklW: double (nullable = true)
 |-- Int: double (nullab

In [27]:
from pyspark.sql.functions import col

df = df.toPandas()

#df.to_csv("data/performnace3.csv", index=False)

In [31]:
# Ordenar por adjusted_score descendente
df_top = df.orderBy(col("adjusted_score").desc())

# Seleccionar columnas clave para visualizar
df_top.select(
    "Player", "Squad", "Pos",
    "adjusted_score", "performance_score", "penalty_score", "Value"
).show(50, truncate=False)  # Puedes ajustar el número de filas

+------------------+---------------+----------------+------------------+-----------------+--------------------+---------+
|Player            |Squad          |Pos             |adjusted_score    |performance_score|penalty_score       |Value    |
+------------------+---------------+----------------+------------------+-----------------+--------------------+---------+
|kevin paredes     |Wolfsburg      |Left Midfield   |22.398752145606597|22.898214        |0.49946219460336405 |5000000.0|
|juan cuadrado     |Atalanta       |Right Midfield  |21.11975944112744 |21.572994        |0.4532347910502959  |1000000.0|
|ademola lookman   |Atalanta       |Second Striker  |20.65285145200088 |20.963503        |0.3106514319102507  |4.0E7    |
|paulo dybala      |Roma           |Second Striker  |20.6037759806492  |20.841646        |0.23787021380880793 |2.0E7    |
|nicola zalewski   |Inter          |Left Midfield   |20.024474804636696|21.233036        |1.2085612366230678  |1.2E7    |
|thomas muller     |Baye

In [36]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

# Definir una ventana por Posición, ordenada por adjusted_score descendente
window_spec = Window.partitionBy("Pos").orderBy(col("adjusted_score").desc())
df_2024_2025 = df.filter(col("Season") == "2024-2025")
# Aplicar ranking
df_ranked = df.withColumn("rank", rank().over(window_spec))

# Filtrar los mejores (rank = 1) por posición
df_top_by_position = df_ranked.filter(col("rank") == 1)

# Mostrar resultados
df_top_by_position.select(
    "Pos", "Player", "Squad", "Season", "adjusted_score", "performance_score", "penalty_score", "Value"
).orderBy("Pos").show(truncate=False)


+------------------+-----------------+---------------+---------+------------------+-----------------+--------------------+---------+
|Pos               |Player           |Squad          |Season   |adjusted_score    |performance_score|penalty_score       |Value    |
+------------------+-----------------+---------------+---------+------------------+-----------------+--------------------+---------+
|Attacking Midfield|florian wirtz    |Leverkusen     |2023-2024|12.950615006391056|13.1680975       |0.2174824896416597  |1.3E8    |
|Central Midfield  |jakub moder      |Brighton       |2024-2025|15.372779846191406|15.37278         |0.0                 |1.2E7    |
|Centre-Back       |giulian biancone |Nott'ham Forest|2022-2023|11.403867721557617|11.403868        |0.0                 |6000000.0|
|Centre-Forward    |antoine griezmann|Atlético Madrid|2022-2023|9.603654310232534 |9.705955         |0.10230024146424449 |2.5E7    |
|Defensive Midfield|joshua kimmich   |Bayern Munich  |2024-2025|11.04

In [54]:
import pandas as pd
df_gk = pd.read_csv("data/keepers/LaLiga_2024_2025_keepers.csv")
df_gk_advanced = pd.read_csv("data/keepersadv/LaLiga_2024_2025_keepersadv.csv")

In [55]:
df_gk.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts',
       'Min', '90s', 'GA', 'GA90', 'SoTA', 'Saves', 'Save%', 'W', 'D', 'L',
       'CS', 'CS%', 'PKatt', 'PKA', 'PKsv', 'PKm', 'Save%.1', 'Matches'],
      dtype='object')

In [56]:
df_gk_advanced.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'GA',
       'PKA', 'FK', 'CK', 'OG', 'PSxG', 'PSxG/SoT', 'PSxG+/-', '/90', 'Cmp',
       'Att', 'Cmp%', 'Att (GK)', 'Thr', 'Launch%', 'AvgLen', 'Att.1',
       'Launch%.1', 'AvgLen.1', 'Opp', 'Stp', 'Stp%', '#OPA', '#OPA/90',
       'AvgDist', 'Matches'],
      dtype='object')